<a href="https://colab.research.google.com/github/coldsober-irene/hackathon1/blob/main/Irene_Nsengumukiza_Hackathon_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installation zone**

In [ ]:
!pip install  -U easynmt

**All required imports**

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json
import numpy as np
from easynmt import EasyNMT
import json # to access languages codes

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  **WEB SCRAPING OF JOB IN RWANDA**  |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [ ]:
class dataMining:
  def __init__(self, jobSource = "Job In Rwanda"):
    """
    this class has a mission to webscrap jobinrwanda.com and finally return the full text of each job.
    """
    self.source = jobSource
    self.jobIdentification = {}
  
  # CREATE SOUP
  def Soup(self, url):
    """BeautifulSoup object will be created in  this function and returned"""
    page = requests.get(url).content
    soup = bs(page, "html.parser")
    return soup

  # JOBS LINKS 
  def get_links(self):
    """
    output link of the jobs linke https://www.jobinrwanda/software-developer
    """
    soup = self.Soup(url = "https://www.jobinrwanda.com/jobs/all")
    company_infoLink = []
    all_jobsLinks = ["https://www.jobinrwanda.com"+link['href'] if "job" in link['href'] 
                     else company_infoLink.append(link.text) 
                     for div in soup.find_all("div", class_= "card-body p-2") 
                     for link in div.find_all("a")
                     ]
    all_jobsLinks = [link for link in all_jobsLinks if link]
    return (all_jobsLinks, company_infoLink)

  # JOBS INFO SUCH AS DEADLINE, JOB TITLE, etc.
  def get_jobInfo(self):
    """
    this function will return a dictionary containing information about the job ex: {'deadline':'2022/10/05', 'sector':'Computer and IT', .....}
    """
    jobLinks = self.get_links()[0]
    info = {}
    
    def refine_info(ls):
      refined_info = {}
      infoZeroPattern = re.compile("\d+.*")
      views = infoZeroPattern.search(ls[0]).group()
      refined_info["views"] = views
      otherInfoPattern = re.compile(":")
      
      for detail in ls[1:]:
        if detail.lower() == "apply":
          pass
        else:
          try:
            detailed = otherInfoPattern.split(detail)
            refined_info[detailed[0]] = detailed[1]
          except IndexError:
            pass
      return refined_info
       
    for index,url in enumerate(jobLinks):
      soup = self.Soup(url)
      pattern = re.compile("\s{2,}")
      job_info = [pattern.sub(" ",li.text.strip().replace("\n", "")) for ul in soup.find_all('ul', class_ = "list-group list-group-flush") 
      for li in ul.find_all('li')]
      # DICTIONANRY OF INFO
      info[index] = refine_info(job_info[:9])
    return info

  def get_description(self):
    """
    full text of the job description will be retrived in this function
    """
    jobLinks = self.get_links()[0]
    all_text= {}
    for index,link in enumerate(jobLinks):
      soup = self.Soup(link)
      # JOB TITLE
      job_title = soup.find('span', class_ = "field field--name-title field--type-string field--label-hidden")
      
      tags_content = soup.find_all('div', class_= "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
      
      for div in tags_content:
        temp = []
        try:
          # GET APPLICATION LINK
          app_link = soup.find("a", class_ = "btn btn-sm btn-success")['href']
          if app_link.startswith("/"):
            app_link = "https://www.jobinrwanda.com"+app_link
          temp.append(app_link)
          
        except TypeError:
          temp.append(np.nan)
        for tag in div.children:
          try:
            temp.append(tag.get_text())
          except (AttributeError, TypeError):
            pass
        temp.insert(0, job_title.text)
        all_text[index] = temp
    return all_text


MERGE PARAGRAPHS OF JOB DESCRPTION TO FORM A VALID TEXT

In [ ]:
def final_textDemo():
  """As the name implies, it is about creating the final text that is ready to be transformed into the dataframe"""
  global info
  # CLASS OBJECT CREATION
  jobInRwanda = dataMining()
  job_text = jobInRwanda.get_description()
  info = jobInRwanda.get_jobInfo()

  final_demo = []
  for index in list(info.keys()):
    info_value = list(info[index].values())
    # ADD COMPANY HIRING
    co_info = jobInRwanda.get_links()[1]
    info_value.append(co_info[index])
    # INCLUDE JOB TITLE
    info_value.append(job_text[index][0])
    # INCLUDE APPLY LINK
    info_value.append(job_text[index][1])
    # JOIN PARAGRAPHS 
    t = '\n'.join(job_text[index][2:])
    g = re.compile("\n|\\xa0").sub("", t)
    info_value.append(g)
    final_demo.append(info_value)
  return final_demo

CREATE THE DATAFRAME OF THE OUTPUT JOB DETAILS

In [ ]:
def makeDataFrame(dataList, columns):
  return pd.DataFrame(dataList, columns = columns)

FILTER THE DATAFRAME TO REMAIN ONLY WITH IT & COMPUTER RELATED JOBS

In [ ]:
df['Is IT Job'] = False
def isIT_Job():
  for index in range(len(df)):
      if "Computer" in df.iloc[index]['Sector'] or "IT" in df.iloc[index]['Sector']:
        df['Is IT Job'][index] = True

ONLY IT & COMPUTER RELATED JOBS

In [ ]:
computer_and_IT_jobs = df[df['Is IT Job'] == True]

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  **WEB  SCRAPING OF UMUCYO WEBSITE**  |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  **JOB DESCRIPTION TRANSLATION ZONE**  |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [ ]:
class translator:
  def __init__(self):
    # INITIALIZE THE MODEL FOR TRANSLATE
    self.model = EasyNMT("opus-mt")
  
  def supportedLangs(self, all_langs, supported):
    with open(all_langs) as f_json:
      content = f_json.read()
    langs = json.loads(content)

    with open(supported) as f2_json:
          content = f2_json.read()
    supplangs = json.loads(content)
    supp_codes = set(list(langs.values())).intersection(set(supplangs))
    supp_keyValue = {}
    for key in langs.keys():
      if langs[key] in supp_codes:
        supp_keyValue[key] = langs[key]
    return supp_keyValue
    

  def translate(self, sentences, targetLang, source = 'en', single = True):
    fail_message = "[FAIL]: destination lang is not a suported language by the model"
    success = 0
    if single:
      try:
        translation = self.model.translate(sentences, source_lang = source, target_lang = targetLang)
        return translation
      except:
        print(fail_message)
    else:
      translated = []
      for target in targetLang:
        try:
          translation = self.model.translate(sentences, source_lang = source, target_lang = target)
          translated.append(translation)
          success += 1
        except:
          print(fail_message)
      return translated, success

# selenium : https://www.browserstack.com/guide/web-scraping-using-selenium-python#:~:text=Selenium%20is%20needed%20in%20order,of%20the%20browser%20being%20used.

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  **API CREATION ZONE**  |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||